Testinate:

In [2]:
from src.assignment2.simulation import HospitalSimulation
from scipy import stats

arr_Times = [14.5, 11.0, 8.0, 6.5, 5.0, 13.0]
len_stay = [2.9, 4.0, 4.5, 1.4, 3.9, 2.2]

H_Sim = HospitalSimulation(stats.expon, stats.expon)



TypeError: __init__() missing 2 required positional arguments: 'number_of_patients' and 'rejected_patients'